In [3]:
# import the necessary packages
import pandas as pd
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2
import matplotlib.pyplot as plt
import serial
import time

In [4]:
def midpoint(ptA, ptB):
	return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def filter_by_color(img: np.ndarray) -> np.ndarray:
    """Filters objects by the blue color in the frame"""
    # Converts the image from BGR color space to HSV
    # load the image, convert it to grayscale, and blur it slightly
    upper = np.array([179, 255, 255], dtype="uint8")
    lower = np.array([0, 0, 180], dtype="uint8")
    
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #hsv[:, :, 2] = cv2.equalizeHist(hsv[:, :, 2])
    # Filter by color
    mask = cv2.inRange(hsv, lower, upper)
    return mask

def remove_noise(img: np.ndarray) -> np.ndarray:
    """Removes the image noise"""
    # Applies binary thresholding
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((15, 15), np.uint8)  # Defines a 5x5 kernel
    # Applies morphological transformation with operator "opening"
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    return opening

def find_background_area(img: np.ndarray) -> np.ndarray:
    """Applies Dilation morphological filter"""
    # sure background area
    kernel = np.ones((40, 40), np.uint8)  # Defines a 3x3 kernel
    sure_bg = cv2.dilate(img, kernel, iterations=2)
    return sure_bg

def find_foreground_area(img: np.ndarray) -> np.ndarray:
    """Finds foreground area"""
    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(img, cv2.DIST_L2, 3)
    _, sure_fg = cv2.threshold(dist_transform, 0.1* dist_transform.max(), 255, 0)
    return sure_fg

def find_contours(sure_bg: np.ndarray, sure_fg: np.ndarray, image: np.ndarray):
    """Finds the contour of each image object"""
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    # Marker labeling
    _, markers = cv2.connectedComponents(sure_fg)
    # Adds one to all labels so that sure background is not 0, but 1
    markers = markers + 1
    # Maks the region of unknown with zero
    markers[unknown == 255] = 0
    markers = cv2.watershed(image, markers)
    return markers

def filter_contours(img):
    """Filter the image contours"""
    kernel = np.ones((5, 5), np.uint8)  # Defines a 2x2 kernel
    # Applies morphological transformation with operator "closing"
    closed = cv2.morphologyEx(
        img.astype(np.uint8), cv2.MORPH_CLOSE, kernel, iterations=2)
    # Applies binary thresholding
    _, thresh = cv2.threshold(closed, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # Finds the contours
    contours, hierarchy = cv2.findContours(
        thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    contours_int = [
        contours[i] for i in range(len(contours)) if hierarchy[0][i][2] < 0
    ]
    return contours_int

In [24]:
#Set the camera and its resolution
cam = cv2.VideoCapture(1, cv2.CAP_DSHOW) #0=front-cam, 1=back-cam
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

#Set the scale value
pixelsPerMetric = 71.52575404

#Open the serial port connection
serialcomm = serial.Serial('COM13', 9600)
serialcomm.timeout = 1

In [ ]:
#Set the initial values for the loop
potato = 1
potato_size_l = []
potatos_sizes_l = []
frame_count = 1
run_n = 1
switch_servo = True

#Create a while loop to capture the video from the camera
while True:
    
    #read the camera frames
    success, frame = cam.read()
     
    #if first frame was sucessfully captured - send the serial signal for Arduino to start belt    
    if(success and frame_count == 1):
        time.sleep(3)
        print(serialcomm.readline().decode('ascii'))
        serialcomm.write('Start'.encode())
        print(serialcomm.readline().decode('ascii'))
    
    #Crop image to remove the sides of the converyor
    cropped = frame[:,200:1000]
    #Threshold the HSV
    img = filter_by_color(cropped)
    
    #Noise removal and fitlering
    img = remove_noise(img)
    sure_bg = find_background_area(img)
    sure_fg = find_foreground_area(img)
    
    #Obtains the contours
    img = find_contours(sure_bg,sure_fg, image= cropped)
    cnts = filter_contours(img = img)
    
    #Evaluate if they are within the limtis
    has_valid_countour = [cv2.contourArea(c)>20000 or cv2.contourArea(c)>288000 for c in cnts]
    
    #If they are go to loop to measure potatoes
    if any(has_valid_countour):
        for c in cnts:
             # if the contour is not sufficiently large, ignore it
            area = cv2.contourArea(c)
            #print(area)
            if area < 20000 or area > 288000:
                continue
            
            #compute the centroid from the countour 
            M = cv2.moments(c)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            
            #If it is the first time running this section difference is 0
            if(run_n == 1):
                diff_cy = 0
                
            #otherwise uses the previous center
            else: 
                diff_cy =  cY_past - cY
            #print(diff_cy)
            
            #Saves the Y centroid coordinate for the next interaction
            cY_past = cY
            
            # compute the rotated bounding box of the contour
            box = cv2.minAreaRect(c)
            box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            # order the points in the contour such that they appear
            # in top-left, top-right, bottom-right, and bottom-left
            # order, then draw the outline of the rotated bounding
            # box
            box = perspective.order_points(box)
            cv2.drawContours(cropped, [box.astype("int")], -1, (0, 255, 0), 2)
            # loop over the original points and draw them
            for (x, y) in box:
                cv2.circle(cropped, (int(x), int(y)), 5, (0, 0, 255), -1)

            # unpack the ordered bounding box, then compute the midpoint
            # between the top-left and top-right coordinates, followed by
            # the midpoint between bottom-left and bottom-right coordinates
            (tl, tr, br, bl) = box
            (tltrX, tltrY) = midpoint(tl, tr)
            (blbrX, blbrY) = midpoint(bl, br)
            # compute the midpoint between the top-left and top-right points,
            # followed by the midpoint between the top-righ and bottom-right
            (tlblX, tlblY) = midpoint(tl, bl)
            (trbrX, trbrY) = midpoint(tr, br)

            if cY > 200 and cY < 500:
                # draw the midpoints on the image
                cv2.circle(cropped, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
                cv2.circle(cropped, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
                cv2.circle(cropped, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
                cv2.circle(cropped, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)
                # draw lines between the midpoints
                cv2.line(cropped, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
                    (255, 0, 255), 2)
                cv2.line(cropped, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
                    (255, 0, 255), 2)

                # compute the Euclidean distance between the midpoints
                dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
                dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
                # if the pixels per metric has not been initialized, then
                # compute it as the ratio of pixels to supplied metric
                # (in this case, inches)
                if pixelsPerMetric is None:
                    pixelsPerMetric = dB / 1

                # compute the size of the object
                dimA = dA / pixelsPerMetric
                dimB = dB / pixelsPerMetric
                # draw the object sizes on the image
                cv2.putText(cropped, "{:.1f}cm".format(dimA),
                    (int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
                    0.65, (255, 255, 255), 2)
                cv2.putText(cropped, "{:.1f}cm".format(dimB),
                    (int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
                    0.65, (255, 255, 255), 2)
                potato_size_l.append([dimA, dimB])
            else:
                #If potato passed the 200 pixel line get all the data collected, take median and classify
                if(cY - 200)< 0 and switch_servo:
                    potato_size = np.median(potato_size_l, axis=0)
                    potato_area = round(potato_size[0]*potato_size[1],2)
                    
                    #Classification is done per area in cm2
                    if potato_area > 58:
                        potato_class = 4
                    if potato_area> 45 and potato_area <=58:
                        potato_class = 3
                    if potato_area> 35 and potato_area <=45:
                        potato_class = 2
                    if potato_area <=34:
                        potato_class = 1
                    
                    #Save results in a list
                    potatos_sizes_l.append([potato, round(potato_size[0], 2), round(potato_size[1],2), potato_area, potato_class])
                    
                    #Write the class on the serial port
                    serialcomm.write(f'{potato_class}'.encode())
                    #time.sleep(0.5)
                    #print(serialcomm.readline().decode('ascii'))
                    switch_servo = False
                    print(potato)
                
                #Once the differnece is negative it got a new potato! Start everything again
                if(diff_cy<-600):
                    potato += 1
                    potato_size_l = []
                    switch_servo = True
                    
            run_n +=1
    
    #Save all the frame for analysis
    cv2.imwrite(f"frames_potato_sorter_5/frame{str(frame_count)}.jpg", cropped)
    cv2.imshow('potato_detection', cropped)
    frame_count += 1
    
    #For a good system, a STOP button has to be implemented. In our case it is the keyboard letter "q"
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
#Once out of the loop close all open connections!
cam.release()
cv2.destroyWindow('potato_detection')
serialcomm.write('Stop'.encode())
print(serialcomm.readline().decode('ascii'))
serialcomm.close()

# And save the data collected
results = pd.DataFrame(potatos_sizes_l)
results.columns = ['Potato', 'Dimension1', 'Dimension2', 'Area', 'Classification']
results.to_csv('seed_potato.csv')

Start of Test


1


In [23]:
results

,Potato,Dimension1,Dimension2,Area,Classification
0,1,5.80,6.31,36.54,2
1,2,5.82,6.30,36.66,2
2,3,5.94,6.36,37.82,2
3,4,5.89,6.28,36.99,2
4,5,5.80,6.36,36.91,2


In [24]:
serialcomm.write('1'.encode())

1